### Imports

In [17]:
# https://huggingface.co/Organika/sdxl-detector#validation-metrics
# The code below fine-tunes the model on the faces and art datasets (separately) and evaluates the model on their validation sets
# https://huggingface.co/blog/fine-tune-vit

import os
import torch
from transformers import AutoImageProcessor, AutoModelForImageClassification
from datasets import load_dataset, load_metric
import numpy as np
import evaluate

print(os.getcwd())
os.chdir('e:\\Projects & Temp\\GitHub\\thesis')

# CUDA check 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

e:\Projects & Temp\GitHub\thesis


### Load Datasets and Model 

In [8]:
# Load model
processor = AutoImageProcessor.from_pretrained("Organika/sdxl-detector")
model = AutoModelForImageClassification.from_pretrained("Organika/sdxl-detector")

# Load data
art_dataset_path = 'archive/datasets/art_512x512'
faces_dataset_path = 'archive/datasets/faces_512x512'

art_ds = load_dataset("imagefolder", data_dir=art_dataset_path) # wish i knew about this when i was doing RQ1
faces_ds = load_dataset("imagefolder", data_dir=faces_dataset_path)

print(faces_ds)

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 12800
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 1600
    })
})


### Pre-Processing

In [11]:
# Transform images to model input
def transform(image_batch):
    inputs = processor(images=image_batch, return_tensors="pt")
    inputs['labels'] = image_batch['labels']
    return inputs

art_ds_transformed = art_ds.with_transform(transform)
faces_ds_transformed = faces_ds.with_transform(transform)

In [13]:
# Turn dicts into tensors
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [18]:
acc_metric = evaluate.load('accuracy')
f1_metric = evaluate.load('f1')

def compute_accuracy(p):
    return acc_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

def compute_f1(p):
    return f1_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

In [ ]:
# remember to write a comment about TrainingArguments and how you ensure that they are the same for the fine-tuned and original pre-trained model 